In [134]:
import sys
sys.path.append('../')
import onnx
# import torch # Activate if use input_to_value
from onnx_utils import *


onnx_model_path = "../models/resnet18.onnx"

In [135]:
# Ejemplo de uso: obtener una lista de los nombres de todas las capas en el modelo ONNX
layer_names = get_layer_names(onnx_model_path)
print("Nombres de las capas:")
print(layer_names)
print(f"Hay un total de {len(layer_names)} capas")

Nombres de las capas:
['/conv1/Conv', '/relu/Relu', '/maxpool/MaxPool', '/layer1/layer1.0/conv1/Conv', '/layer1/layer1.0/relu/Relu', '/layer1/layer1.0/conv2/Conv', '/layer1/layer1.0/Add', '/layer1/layer1.0/relu_1/Relu', '/layer1/layer1.1/conv1/Conv', '/layer1/layer1.1/relu/Relu', '/layer1/layer1.1/conv2/Conv', '/layer1/layer1.1/Add', '/layer1/layer1.1/relu_1/Relu', '/layer2/layer2.0/conv1/Conv', '/layer2/layer2.0/relu/Relu', '/layer2/layer2.0/conv2/Conv', '/layer2/layer2.0/downsample/downsample.0/Conv', '/layer2/layer2.0/Add', '/layer2/layer2.0/relu_1/Relu', '/layer2/layer2.1/conv1/Conv', '/layer2/layer2.1/relu/Relu', '/layer2/layer2.1/conv2/Conv', '/layer2/layer2.1/Add', '/layer2/layer2.1/relu_1/Relu', '/layer3/layer3.0/conv1/Conv', '/layer3/layer3.0/relu/Relu', '/layer3/layer3.0/conv2/Conv', '/layer3/layer3.0/downsample/downsample.0/Conv', '/layer3/layer3.0/Add', '/layer3/layer3.0/relu_1/Relu', '/layer3/layer3.1/conv1/Conv', '/layer3/layer3.1/relu/Relu', '/layer3/layer3.1/conv2/Conv'

In [136]:
class Layer:
    def __init__(self, onnx_model_path, layer_name):
        self.model = onnx.load(onnx_model_path)
        self.graph = self.model.graph
        self.layer_name = layer_name
        self.node = self._find_node_by_name(layer_name)

    def _find_node_by_name(self, name):
        for node in self.graph.node:
            if node.name == name:
                return node
        raise ValueError(f"No se encontró una capa con el nombre {name}")

    def get_inputs(self):
        return self.node.input[0]

    def get_outputs(self):
        return self.node.output

    def get_attribute(self, attr_name):
        for attr in self.node.attribute:
            if attr.name == attr_name:
                return attr
        return None

In [137]:
class ConvLayer(Layer):
    def __init__(self, onnx_model_path, layer_name):
        super().__init__(onnx_model_path, layer_name)

    def get_weights(self):
        for initializer in self.graph.initializer:
            if initializer.name == self.node.input[1]:  # Suponiendo que el segundo input es el peso
                return initializer.name

    def get_bias(self):
        for initializer in self.graph.initializer:
            if initializer.name == self.node.input[2]:  # Suponiendo que el tercer input es el sesgo
                b = "" + initializer.name
                return b

    def get_kernel_size(self):
        return self.get_attribute('kernel_shape').ints

    def get_stride(self):
        return self.get_attribute('strides').ints

    def get_padding(self):
        return self.get_attribute('pads').ints

    def get_groups(self):
        return self.get_attribute('group').i

In [138]:
# Ejemplo de uso:

conv_layer = ConvLayer(onnx_model_path, layer_names[0])
print("Conv Layer Weights:", conv_layer.get_weights())
print("Conv Layer Bias:", conv_layer.get_bias())

print("Conv Layer Kernel Size:", conv_layer.get_kernel_size())
print("Conv Layer Stride:", conv_layer.get_stride())
print("Conv Layer Padding:", conv_layer.get_padding())
print("Conv Layer Groups:", conv_layer.get_groups())

# Obtener los inputs y outputs de la capa convolucional
print(f"Inputs de la capa: {conv_layer.get_inputs()}")
print(f"Outputs de la capa: {conv_layer.get_outputs()}")

Conv Layer Weights: onnx::Conv_193
Conv Layer Bias: onnx::Conv_194
Conv Layer Kernel Size: [7, 7]
Conv Layer Stride: [2, 2]
Conv Layer Padding: [3, 3, 3, 3]
Conv Layer Groups: 1
Inputs de la capa: input.1
Outputs de la capa: ['/conv1/Conv_output_0']


In [139]:
class ReluLayer(Layer):
    def __init__(self, onnx_model_path, layer_name):
        super().__init__(onnx_model_path, layer_name)

In [140]:
relu_layer = ReluLayer(onnx_model_path, layer_names[4])
print("ReLU Layer Inputs:", relu_layer.get_inputs())
print("ReLU Layer Outputs:", relu_layer.get_outputs())

ReLU Layer Inputs: /layer1/layer1.0/conv1/Conv_output_0
ReLU Layer Outputs: ['/layer1/layer1.0/relu/Relu_output_0']


In [141]:
class MaxPoolLayer(Layer):
    def __init__(self, onnx_model_path, layer_name):
        super().__init__(onnx_model_path, layer_name)

    def get_kernel_size(self):
        return self.get_attribute('kernel_shape').ints

    def get_stride(self):
        return self.get_attribute('strides').ints

    def get_padding(self):
        return self.get_attribute('pads').ints

In [142]:
maxpool_layer = MaxPoolLayer(onnx_model_path, layer_names[2])
print("MaxPool Layer Kernel Size:", maxpool_layer.get_kernel_size())
print("MaxPool Layer Stride:", maxpool_layer.get_stride())
print("MaxPool Layer Padding:", maxpool_layer.get_padding())

print("MaxPool Inputs:", maxpool_layer.get_inputs())
print("MaxPool Outputs:", maxpool_layer.get_outputs())

MaxPool Layer Kernel Size: [3, 3]
MaxPool Layer Stride: [2, 2]
MaxPool Layer Padding: [1, 1, 1, 1]
MaxPool Inputs: /relu/Relu_output_0
MaxPool Outputs: ['/maxpool/MaxPool_output_0']


In [143]:
class AddLayer(Layer):
    def __init__(self, onnx_path, layer_name):
        super().__init__(onnx_path, layer_name)

In [144]:
add_layer = AddLayer(onnx_model_path, layer_names[44])
print("Add Layer Inputs:", add_layer.get_inputs())
print("Add Layer Outputs:", add_layer.get_outputs())

Add Layer Inputs: /layer4/layer4.1/conv2/Conv_output_0
Add Layer Outputs: ['/layer4/layer4.1/Add_output_0']


In [145]:
class GlobalAveragePoolLayer(Layer):
    def __init__(self, onnx_path, layer_name):
        super().__init__(onnx_path, layer_name)

In [146]:
global_average_pool_layer = GlobalAveragePoolLayer(onnx_model_path, layer_names[46])
print("GlobalAveragePool Layer Inputs:", global_average_pool_layer.get_inputs())
print("GlobalAveragePool Layer Outputs:", global_average_pool_layer.get_outputs())

GlobalAveragePool Layer Inputs: /layer4/layer4.1/relu_1/Relu_output_0
GlobalAveragePool Layer Outputs: ['/avgpool/GlobalAveragePool_output_0']


In [147]:
class FlattenLayer(Layer):
    def __init__(self, onnx_path, layer_name):
        super().__init__(onnx_path, layer_name)

In [148]:
flatten_layer = FlattenLayer(onnx_model_path, layer_names[47])
print("Flatten Layer Inputs:", flatten_layer.get_inputs())
print("Flatten Layer Outputs:", flatten_layer.get_outputs())

Flatten Layer Inputs: /avgpool/GlobalAveragePool_output_0
Flatten Layer Outputs: ['/Flatten_output_0']


In [149]:
class GemmLayer(Layer):
    def __init__(self, onnx_path, layer_name):
        super().__init__(onnx_path, layer_name)

    def get_weights(self):
        for initializer in self.graph.initializer:
            if initializer.name == self.node.input[1]:  # Suponiendo que el segundo input es el peso
                return initializer.name

    def get_bias(self):
        for initializer in self.graph.initializer:
            if initializer.name == self.node.input[2]:  # Suponiendo que el tercer input es el sesgo
                return initializer.name

In [150]:
gemm_layer = GemmLayer(onnx_model_path, layer_names[48])
print("Gemm Layer Inputs:", gemm_layer.get_inputs())
print("Gemm Layer Outputs:", gemm_layer.get_outputs())
print("Gemm Layer Weights:", gemm_layer.get_weights())
print("Gemm Layer Bias:", gemm_layer.get_bias())

Gemm Layer Inputs: /Flatten_output_0
Gemm Layer Outputs: ['191']
Gemm Layer Weights: fc.weight
Gemm Layer Bias: fc.bias
